# Copying over all forgotten files on the corr nodes

In [11]:
import time
import numpy as np
from subprocess import check_output
from dsautils.dsa_store import DsaStore
from dsautils import cnf
ETCD = DsaStore()

In [18]:
myconf = cnf.Conf()
params = myconf.get('corr')
corrnames = list(params['ch0'].keys())
filenames = [None]*len(corrnames)

In [19]:
for i, corr in enumerate(corrnames):
    try:
        filelist = check_output([
            "rsync",
            "--list-only",
            "{0}.sas.pvt:~/data/*.hdf5".format(corr),
            "--exclude",
            "*incomplete.hdf5"
        ]).decode("utf-8")
        filelist = filelist.split('\n')
        while '' in filelist:
            filelist.remove('')
        filenames[i] = sorted(file.split(' ')[-1] for file in filelist)
    except:
        filenames[i] = []
for fn in filenames:
    fn += ['9999']

In [8]:
sum([len(fn) for fn in filenames])-len(filenames)

4

In [9]:
filenames

[['2021-03-26T00:27:50.hdf5',
  '2021-03-26T00:43:16.hdf5',
  '2021-03-26T00:58:18.hdf5',
  '2021-03-26T01:13:20.hdf5',
  '9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999'],
 ['9999']]

In [10]:
while sum([len(fn) for fn in filenames]) > len(filenames):
    nextfiles = [fn[0] for fn in filenames]
    file = min(nextfiles)
    corridx = nextfiles.index(file)
    ETCD.put_dict(
        '/cmd/cal',
        {'cmd': 'rsync',
         'val': {'hostname': corrnames[corridx],
                 'filename': '/home/ubuntu/data/{0}'.format(file)
        }
        }
    )
    filenames[corridx].remove(file)
    time.sleep(1)